In [5]:
import os
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

In [8]:
#os.getenv('OPENAI_API_KEY')

### OpenAI Basic Chat API

In [9]:
import json
from openai import OpenAI

In [10]:
client = OpenAI()

In [11]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    n=1,
    temperature=0.5,
    messages=[
        {
            "role": "user",
            "content": "Who won first cricket worldcup?",
        }
    ]
)
response.choices[0].message.content

'The first Cricket World Cup was won by the West Indies in 1975.'

Check tokens calculation [https://platform.openai.com/tokenizer](https://platform.openai.com/tokenizer)

Check Pricing for tokens [https://openai.com/api/pricing/](https://openai.com/api/pricing/)

### Moderation - Flagging inappropriate content

In [28]:
response = client.moderations.create(input="i want to starve myself?")
response.results[0]

print("\nis Flagged: ", response.results[0].flagged, '\n')

for name, value in dict(response.results[0].categories).items():
    print(name, value)


is Flagged:  True 

harassment False
harassment_threatening False
hate False
hate_threatening False
self_harm True
self_harm_instructions False
self_harm_intent True
sexual False
sexual_minors False
violence False
violence_graphic False
self-harm True
sexual/minors False
hate/threatening False
violence/graphic False
self-harm/intent True
self-harm/instructions False
harassment/threatening False


In [29]:
response = client.moderations.create(input="How to make some explosive chemicals?")
response.results[0]

print("\nis Flagged: ", response.results[0].flagged, '\n')

for name, value in dict(response.results[0].categories).items():
    print(name, value)


is Flagged:  False 

harassment False
harassment_threatening False
hate False
hate_threatening False
self_harm False
self_harm_instructions False
self_harm_intent False
sexual False
sexual_minors False
violence False
violence_graphic False
self-harm False
sexual/minors False
hate/threatening False
violence/graphic False
self-harm/intent False
self-harm/instructions False
harassment/threatening False


### Text to Image

In [36]:
response = client.images.generate(
  model="dall-e-2",
  prompt="a white siamese cat",
  size="256x256",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= image_url)

##### The size of the generated images. Must be one of 256x256 , 512x512 , or 1024x1024 for dall-e-2 . Must be one of 1024x1024 , 1792x1024 , or 1024x1792 for dall-e-3 models

### Function Calling
connect large language models to external tools

In [7]:
prompt = f'''
please extract the following information from the given text in JSON format.

year
winner_team
runner_up_team

Information:
First cricket worldcup was played in 1975 between West Indies and Australia. This worldcup was won by West Indies.
'''
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ]
)
json.loads(response.choices[0].message.content)

{'year': 1975, 'winner_team': 'West Indies', 'runner_up_team': 'Australia'}

#### Same as above but using Function Calling

In [8]:
functions = [
    {
        "name": "extract_details",
        "parameters": {
            "type": "object",
            "properties": {
                "year": {"type": "integer"},
                "winner_team": {"type": "string"},
                "runner_up_team": {"type": "string"},
            }
        }
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    functions=functions,
    function_call="auto",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ]
)
json.loads(response.choices[0].message.function_call.arguments)

{}

### Inline function call

In [9]:
from datetime import datetime, timedelta

In [10]:
prompt = "when is the next cricket match between pakistan and india?"

def get_next_match(team1: str, team2: str):
    # dummy function to return next match date between two teams. this could be replaced with espncricinfo api to get real data
    return json.dumps({"team1": team1, "team2": team2, "next_match_date": str(datetime.today() + timedelta(days=7))})

functions = [
    {
        "name": "get_next_match",
        "parameters": {
            "type": "object",
            "properties": {
                "team1": {"type": "string"},
                "team2": {"type": "string"},
                "next_match_date": {"type": "string"},
            },
            "required": ["team1", "team2"]
        }
    }
]

response1 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "user", "content": prompt}
    ]
)
params = json.loads(response1.choices[0].message.function_call.arguments)
get_next_match_func = eval(response1.choices[0].message.function_call.name)(**params)

response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "user", "content": prompt},
        {"role": "function", "name": response1.choices[0].message.function_call.name, "content": get_next_match_func}
    ]
)

response2.choices[0].message.content

'The next cricket match between Pakistan and India is scheduled to take place on June 26, 2024 at 01:45:14 UTC.'